<a href="https://colab.research.google.com/github/cabamarcos/P1-RRNN/blob/main/P1_RRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 1 Redes de Neuronas

## 1. Introducción

## 2. Programación del modelo lineal Adaline

In [1]:
import numpy as np
import pandas as pd
import random

In [ ]:
class Adaline:
  def __init__(learning_rate, n_iter = 100000):
    self.learning_rate = learning_rate
    self.n_iter = n_iter

  def fit(self, X, y):




In [4]:
# Cargamos lso datos del 2011 en un pandas dataframe
datos_2011 = pd.read_csv("gt_2011.csv")
datos_2011

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,4.5878,1018.7,83.675,3.5758,23.979,1086.2,549.83,134.67,11.898,0.32663,81.952
1,4.2932,1018.3,84.235,3.5709,23.951,1086.1,550.05,134.67,11.892,0.44784,82.377
2,3.9045,1018.4,84.858,3.5828,23.990,1086.5,550.19,135.10,12.042,0.45144,83.776
3,3.7436,1018.3,85.434,3.5808,23.911,1086.5,550.17,135.03,11.990,0.23107,82.505
4,3.7516,1017.8,85.182,3.5781,23.917,1085.9,550.00,134.67,11.910,0.26747,82.028
...,...,...,...,...,...,...,...,...,...,...,...
7406,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,111.61,10.400,4.51860,79.559
7407,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,111.78,10.433,4.84700,79.917
7408,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,110.19,10.483,7.96320,90.912
7409,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,110.74,10.533,6.24940,93.227
